|Sequência|Detalhamento|
|---|---|
|01|Objetivo: Ingestão de Arquivos CSV|
|02|Data Criação: 25/09/2023|
|03|Desenvolvida por: Gabriel Quintella|


In [0]:
%run /estudos/libs/bibliotecas

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import * #StructType, StructField, StringType
from faker import Faker
from datetime import datetime
import pandas as pd
from pandas import DataFrame
import os

spark = SparkSession.builder.appName("ingestao-pyspark-csv").getOrCreate()

In [0]:
pip install faker

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
clientes_fake = (
    spark.createDataFrame( geracao_dados_fake(1000000)
                              )
)

In [0]:
numero_linhas = clientes_fake.count()

numero_colunas = len(clientes_fake.dtypes)

print(f"Qtd Linhas: {numero_linhas}" )
print(f"Qtd Colunas: {numero_colunas}" )


Qtd Linhas: 1000000
Qtd Colunas: 7


In [0]:
pathArquivo = "/Storage/dados_fake/cliente/"
cria_diretorio(pathArquivo)

Quintellão, o diretório /Storage/dados_fake/cliente/ já existe!


In [0]:
(
    clientes_fake
      .write.option("header", "true").option('delimiter',';')
      .format("csv")
      .mode("overwrite")
      .save(f"{pathArquivo}/cliente_fake.csv")
)

In [0]:

arquivo = f"{pathArquivo}/cliente_fake.csv"

df = spark.read.option('delimiter',';').option("header",True).csv(arquivo)


In [0]:

df = spark.read.options(
    delimiter=';'
    ,header='True'
    ,inferSchema='True'
    ,encoding='utf-8'
).csv(arquivo)


In [0]:
#StructField(NOME DA COLUNA , TIPO DE DADO , PODE OU NÃO ACEITAR VALORES NULOS )

schema = StructType([
StructField('NomFunc', StringType() , True )
,StructField('DscCargo', StringType() , True )
,StructField('NomCidade', StringType() , True )
,StructField('NumTelefone', StringType() , True )
,StructField('NomEmail', StringType() , True )
,StructField('DtNasc', DateType() , True )
,StructField('NumIdade', IntegerType() , True )
])

df = (spark.read.option('delimiter',';')
        .option("header",True)
        .schema(schema)
        .csv(arquivo))



In [0]:
df.describe()

Out[36]: DataFrame[summary: string, NomFunc: string, DscCargo: string, NomCidade: string, NumTelefone: string, NomEmail: string, NumIdade: string]

In [0]:
df.describe().show()

+-------+--------------+------------------+-----------+---------------+--------------------+------------------+
|summary|       NomFunc|          DscCargo|  NomCidade|    NumTelefone|            NomEmail|          NumIdade|
+-------+--------------+------------------+-----------+---------------+--------------------+------------------+
|  count|       1000000|           1000000|    1000000|        1000000|             1000000|           1000000|
|   mean|          null|              null|       null|           null|                null|         26.383086|
| stddev|          null|              null|       null|           null|                null|15.511504882614972|
|    min|Agatha Almeida|      Acompanhante|Afeganistão|(011) 0000-6439| aalmeida@almeida.br|                 0|
|    max|Yuri das Neves|Árbitro e mediador|      Índia|   84 9999-9582|zvieira@yahoo.com.br|                53|
+-------+--------------+------------------+-----------+---------------+--------------------+------------

In [0]:
df.describe(['NumIdade']).show()

+-------+------------------+
|summary|          NumIdade|
+-------+------------------+
|  count|           1000000|
|   mean|         26.383086|
| stddev|15.511504882614972|
|    min|                 0|
|    max|                53|
+-------+------------------+



In [0]:
df.show(5, truncate=False)

+---------------------+-----------------------------------+-----------+-------------------+------------------------+----------+--------+
|NomFunc              |DscCargo                           |NomCidade  |NumTelefone        |NomEmail                |DtNasc    |NumIdade|
+---------------------+-----------------------------------+-----------+-------------------+------------------------+----------+--------+
|Enrico Cunha         |Assessor de imprensa               |Filipinas  |+55 (031) 5652-3864|gustavopeixoto@costa.com|1998-04-05|25      |
|Erick Viana          |Astrofísico                        |Suazilândia|(084) 7768 3973    |gsilveira@bol.com.br    |2005-01-25|18      |
|Giovanna da Conceição|Técnico em higiene dentária        |Costa Rica |+55 51 4621 5457   |carolina87@bol.com.br   |1993-09-01|30      |
|Bernardo Vieira      |Viveirista                         |Tokelau    |+55 (011) 9316-7024|isilveira@melo.com      |1998-05-11|25      |
|Caroline Melo        |Broker/Corretor da

In [0]:
df.printSchema()

root
 |-- NomFunc: string (nullable = true)
 |-- DscCargo: string (nullable = true)
 |-- NomCidade: string (nullable = true)
 |-- NumTelefone: string (nullable = true)
 |-- NomEmail: string (nullable = true)
 |-- DtNasc: date (nullable = true)
 |-- NumIdade: integer (nullable = true)



In [0]:
df.head(5)

Out[41]: [Row(NomFunc='Enrico Cunha', DscCargo='Assessor de imprensa', NomCidade='Filipinas', NumTelefone='+55 (031) 5652-3864', NomEmail='gustavopeixoto@costa.com', DtNasc=datetime.date(1998, 4, 5), NumIdade=25),
 Row(NomFunc='Erick Viana', DscCargo='Astrofísico', NomCidade='Suazilândia', NumTelefone='(084) 7768 3973', NomEmail='gsilveira@bol.com.br', DtNasc=datetime.date(2005, 1, 25), NumIdade=18),
 Row(NomFunc='Giovanna da Conceição', DscCargo='Técnico em higiene dentária', NomCidade='Costa Rica', NumTelefone='+55 51 4621 5457', NomEmail='carolina87@bol.com.br', DtNasc=datetime.date(1993, 9, 1), NumIdade=30),
 Row(NomFunc='Bernardo Vieira', DscCargo='Viveirista', NomCidade='Tokelau', NumTelefone='+55 (011) 9316-7024', NomEmail='isilveira@melo.com', DtNasc=datetime.date(1998, 5, 11), NumIdade=25),
 Row(NomFunc='Caroline Melo', DscCargo='Broker/Corretor da bolsa de valores', NomCidade='Itália', NumTelefone='(041) 7741 9403', NomEmail='esouza@aragao.org', DtNasc=datetime.date(2009, 9, 

In [0]:
df.tail(5)

Out[42]: [Row(NomFunc='Lucas Martins', DscCargo='Roteirista', NomCidade='Papua-Nova Guiné', NumTelefone='+55 (084) 5504 2539', NomEmail='diasjoao-pedro@da.net', DtNasc=datetime.date(2012, 3, 21), NumIdade=11),
 Row(NomFunc='Davi Costa', DscCargo='Presidente da República', NomCidade='Líbano', NumTelefone='(084) 3332 5766', NomEmail='pdias@farias.org', DtNasc=datetime.date(1971, 10, 1), NumIdade=52),
 Row(NomFunc='Vicente da Conceição', DscCargo='Passador', NomCidade='Butão', NumTelefone='+55 51 1593 6073', NomEmail='razevedo@gmail.com', DtNasc=datetime.date(1979, 1, 5), NumIdade=44),
 Row(NomFunc='Clarice Porto', DscCargo='Podólogo', NomCidade='Antártica', NumTelefone='21 9890 7839', NomEmail='aragaoalana@hotmail.com', DtNasc=datetime.date(1993, 10, 20), NumIdade=29),
 Row(NomFunc='Noah da Paz', DscCargo='Laboratorista', NomCidade='Roménia', NumTelefone='0300 496 9738', NomEmail='marcela92@melo.br', DtNasc=datetime.date(1997, 4, 5), NumIdade=26)]

In [0]:
df.show()

+--------------------+--------------------+--------------+-------------------+--------------------+----------+--------+
|             NomFunc|            DscCargo|     NomCidade|        NumTelefone|            NomEmail|    DtNasc|NumIdade|
+--------------------+--------------------+--------------+-------------------+--------------------+----------+--------+
|        Enrico Cunha|Assessor de imprensa|     Filipinas|+55 (031) 5652-3864|gustavopeixoto@co...|1998-04-05|      25|
|         Erick Viana|         Astrofísico|   Suazilândia|    (084) 7768 3973|gsilveira@bol.com.br|2005-01-25|      18|
|Giovanna da Conce...|Técnico em higien...|    Costa Rica|   +55 51 4621 5457|carolina87@bol.co...|1993-09-01|      30|
|     Bernardo Vieira|          Viveirista|       Tokelau|+55 (011) 9316-7024|  isilveira@melo.com|1998-05-11|      25|
|       Caroline Melo|Broker/Corretor d...|        Itália|    (041) 7741 9403|   esouza@aragao.org|2009-09-04|      14|
|      Alexia Correia|    Gerente de ban

In [0]:
display(df)

NomFunc,DscCargo,NomCidade,NumTelefone,NomEmail,DtNasc,NumIdade
Enrico Cunha,Assessor de imprensa,Filipinas,+55 (031) 5652-3864,gustavopeixoto@costa.com,1998-04-05,25
Erick Viana,Astrofísico,Suazilândia,(084) 7768 3973,gsilveira@bol.com.br,2005-01-25,18
Giovanna da Conceição,Técnico em higiene dentária,Costa Rica,+55 51 4621 5457,carolina87@bol.com.br,1993-09-01,30
Bernardo Vieira,Viveirista,Tokelau,+55 (011) 9316-7024,isilveira@melo.com,1998-05-11,25
Caroline Melo,Broker/Corretor da bolsa de valores,Itália,(041) 7741 9403,esouza@aragao.org,2009-09-04,14
Alexia Correia,Gerente de banco,Reino Unido,(051) 9421 1902,maria-vitoriamartins@yahoo.com.br,2012-01-01,11
Benjamin Ferreira,Orçamentista,Omã,+55 (061) 2383 5404,kaiqueoliveira@nunes.br,1987-08-09,36
Brenda da Rosa,Cantor,Senegal,+55 81 8760-4838,da-luzotavio@silveira.br,2009-03-29,14
Bryan Ferreira,Vigilante noturno/diurno,Suazilândia,21 0155-6830,ana-laura86@hotmail.com,1987-06-27,36
Anthony Lima,Cabo,Montenegro,51 5872 5913,monteirothales@hotmail.com,2013-03-18,10


In [0]:
df.columns

Out[49]: ['NomFunc',
 'DscCargo',
 'NomCidade',
 'NumTelefone',
 'NomEmail',
 'DtNasc',
 'NumIdade']